# Part 1: Introduction to SpatialData & Interactive Exploration

**Instructor:** Tim Treis
**Time:** 40 minutes

---

Welcome to the workshop! In this first notebook, we will introduce the core concepts behind the `SpatialData` framework.

**Goals:**
1. Understand the different components of a spatial dataset (Images, Labels, Shapes, Points).
2. Learn how the `SpatialData` object acts as a unified container for these components.
3. Gain hands-on experience exploring complex spatial data interactively with `napari`.

## What is a `SpatialData` object?

Modern spatial omics experiments generate diverse types of data. For example, a single experiment might produce:

*   A high-resolution histology image (**Image**).
*   Segmentation masks defining where the cells are (**Labels**).
*   Polygons outlining cell boundaries (**Shapes**).
*   The locations of individual RNA molecules (**Points**).
*   A table of gene counts per cell (**Table**).

The `SpatialData` framework, part of the `scverse` ecosystem, provides a single, standardized object to hold all these different *elements* together in a coordinated way.

<img src="../resources/elements.png" alt="spatialdata design is modular" style="max-width: 800px;">

## Loading our First Dataset: A Xenium Example

To begin, we will load a pre-processed dataset into a `SpatialData` object. For this workshop, we will be working with the `.zarr` format, which is a modern, high-performance storage format ideal for large scientific data.

Our first dataset is a `Xenium` experiment, which is a great example because it contains all the different types of spatial elements.

In [ ]:
# For cleaner output

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import spatialdata as sd
from pathlib import Path

# Define the path to our data directory
# Note: This path is relative to the repository's root directory
_DATA_DIR_PATH = Path("../data/")
_XENIUM_PATH = _DATA_DIR_PATH / "xenium_lung_cancer_subset.zarr"
_MERFISH_PATH = _DATA_DIR_PATH / "merfish_subset.zarr"

print(f"Loading data from '{_XENIUM_PATH}'")

# Load the pre-processed Xenium dataset
sdata_xenium = sd.read_zarr(_XENIUM_PATH)

Let's inspect the `SpatialData` object we just created. We can directly access the contained images by their name.

In [ ]:
sdata_xenium

### Images 

We see that the contained **Images** are represented as a [`xarray.DataTree`](https://docs.xarray.dev/en/latest/generated/xarray.DataTree.html) which we use for **multiscale images**. These are image pyramids that store the same image at different resolutions, which is key for fast visualization of very large images.

![image pyramid](../resources/image_pyramid.png)

In [ ]:
sdata_xenium.images["morphology_focus"]

In [ ]:
# We can also directly access and plot the data contained

import matplotlib.pyplot as plt

plt.imshow(sdata_xenium.images["morphology_focus"]["scale4"].image.transpose("y", "x", "c"))


### Labels

Similarly, segmentation masks (**Labels**) are also represented as images and can be accessed the same way.

In [ ]:
plt.imshow(sdata_xenium.labels["nucleus_labels"]["scale4"].image.transpose("y", "x"))

### Points

The **Points** objects are represented as a [`Dask.DataFrame`](https://docs.dask.org/en/stable/dataframe.html) which allows us to lazily load the points, meaning that not all of them will be loaded into memory immedediately.

In [ ]:
sdata_xenium.points["transcripts"]

In [ ]:
# We can load them on-demand though.

sdata_xenium.points["transcripts"].compute()

### Shapes

The segmentation masks (**Shapes**) are stored as [`geopandas.GeoDataFrame`](https://geopandas.org/en/v1.1.0/docs/reference/api/geopandas.GeoDataFrame.html).

In [ ]:
sdata_xenium.shapes["cell_boundaries"]

In [ ]:
sdata_xenium.shapes["cell_boundaries"].geometry[0]

### Tables

Lastly, tabular data, for example, the table of gene counts, is stored as an [`anndata.AnnData`](https://anndata.readthedocs.io/en/latest/generated/anndata.AnnData.html) object that many of you may be familiar with from single-cell analysis.

In [ ]:
sdata_xenium.tables["table"]

### Interactive Exploration with `napari`

While printing the object summary is useful, the best way to understand spatial data is to *see* it. We use **Napari**, a fast, interactive, multi-dimensional image viewer, to explore our `SpatialData` objects.

The `napari-spatialdata` plugin provides the bridge between our data and the viewer. To demonstrate its features, we'll load a `MERFISH` dataset, another high-resolution imaging-based technology.

In [ ]:
sdata_merfish = sd.read_zarr(_MERFISH_PATH)
sdata_merfish

#### Your Turn: Launching Napari

Now, let's launch the interactive viewer. 

**Instructions:**
1. Uncomment and run the code cell below.
2. A new window for Napari should appear. This may take a few seconds.
3. Follow along as we tour the interface and explore the data layers.

**Troubleshooting**
`napari` support within Docker containers is limited. If you are not able to run the notebook with the current Docker configuration, please skip the cell below and follow the demo from the instructor. To run `napari` locally without Docker, please use `mamba`, as explained in [the section "Using conda / mamba" from this guide](https://github.com/PMBio/spatialdata-workshops/tree/main?tab=readme-ov-file#using-conda--mamba).

In [ ]:
from napari_spatialdata import Interactive

# run napari
Interactive(sdata_merfish)

<div style="border: 1px solid #4CAF50; border-left-width: 15px; padding: 10px; background-color: #F0FFF0; color: black;">
    <strong>Tip:</strong>
    <p>You can also view a .zarr file from your terminal with:<br><code>python -m napari_spatialdata view ../data/merfish_subset.zarr</code>.</p>
</div>

### Tour of the `napari-spatialdata` Interface

The Napari window is composed of several key parts. The `napari-spatialdata` plugin adds the panels highlighted in green.

![overview of the napari-spatialdata interface](../resources/napari_spatialdata0.jpg)

**Key areas to know:**
- **Layers list (bottom-left):** This is where you can see all the data layers (images, shapes, points) and toggle their visibility, change their order, or adjust their opacity.
- **`spatialdata` viewer (top-right):** This shows you which coordinate systems are available and lets you select elements to view.
- **`spatialdata` table annotation (bottom-right):** This powerful panel lets you color your spatial elements (like cells) by any value in the associated `AnnData` table.

Here is an example of adding elements to the viewer and interacting with them.

![adding elements](../resources/napari_spatialdata1.gif)
![interaction with napari](../resources/napari_spatialdata2.gif)

The real power comes from coloring your spatial elements by their associated data. Here, we color cells by different annotations stored in the `AnnData` table, including categorical variables (`region`) and continuous variables (gene expression).

![showing annotations](../resources/napari_spatialdata3.gif)

The various selectors on the right side of the interface reflect the structure of the `AnnData` object that annotates our spatial elements. The dropdown menus map directly to the `.obs`, `.layers`, and `.obsm` slots of the table.

<img src="https://raw.githubusercontent.com/scverse/anndata/main/docs/_static/img/anndata_schema.svg" width="500" height="500">

#### Advanced Tip: Coloring Points by their Own Attributes

Most of the time, you will color cells/spots by data in an `AnnData` table. However, it's also possible to color `Points` or `Shapes` by columns in their own internal dataframe. For example, individual transcripts might have a `quality_value` column.

The "Dataframe columns" list widget at the bottom of the annotation panel allows you to do this.

![showing annotations points](../resources/napari_spatialdata4.jpg)


<div style="border: 1px solid #4CAF50; border-left-width: 15px; padding: 10px; background-color: #F0FFF0; color: black;">
    <strong>Summary:</strong>
    <p>This concludes our introduction to the <code>SpatialData</code> object and interactive visualization. In the next notebook, we'll learn how to create static, publication-ready plots using <a href="https://spatialdata.scverse.org/projects/plot/en/latest/"><code>spatialdata-plot</code></a>.</p>
</div>

